In this notebook, we use line profiling to see what the bottleneck is in TOI parsing. (See [here](http://mortada.net/easily-profile-python-code-in-jupyter.html) for an example.)

In [1]:
import re
import pandas as pd
from scrapenhl2.scrape import parse_toi, schedules, players, scrape_toi, general_helpers as helpers
%load_ext line_profiler

The first thing we should try is the overall parsing.

In [2]:
%timeit parse_toi.parse_game_toi_from_html(2012, 20001, True)

1 loop, best of 3: 4.54 s per loop


In [3]:
%lprun -f parse_toi.parse_game_toi_from_html parse_toi.parse_game_toi_from_html(2012, 20001, True)

![alt text](parse_game_toi_from_html.png "Parse Game TOI")

Next, we'll try read_shifts_from_html_pages.

In [4]:
season = 2012
game = 20001
from scrapenhl2.scrape import schedules
gameinfo = schedules.get_game_data_from_schedule(season, game)

In [5]:
%%timeit 
parse_toi.read_shifts_from_html_pages(
    scrape_toi.get_raw_html_toi(season, game, 'H'), scrape_toi.get_raw_html_toi(season, game, 'R'),
    gameinfo['Home'], gameinfo['Road'], season, game)

1 loop, best of 3: 4.93 s per loop


In [6]:
%lprun -f parse_toi.read_shifts_from_html_pages parse_toi.read_shifts_from_html_pages(scrape_toi.get_raw_html_toi(season, game, 'H'), scrape_toi.get_raw_html_toi(season, game, 'R'),gameinfo['Home'], gameinfo['Road'], season, game)

![alt text](read_shifts_from_html_pages1.png "Read shifts 1")

![alt text](read_shifts_from_html_pages2.png "Read shifts 2")

It looks like the HTML parser is not bad. The bigger bottleneck is the manipulations at the end.

In [7]:
rawtoi1 = scrape_toi.get_raw_html_toi(season, game, 'H')
rawtoi2 = scrape_toi.get_raw_html_toi(season, game, 'R')
teamid1 = gameinfo['Home']
teamid2 = gameinfo['Road']

from html_table_extractor.extractor import Extractor
dflst = []
for rawtoi, teamid in zip((rawtoi1, rawtoi2), (teamid1, teamid2)):
    extractor = Extractor(rawtoi)
    extractor.parse()
    tables = extractor.return_list()

    ids = [None for _ in range(len(tables))]
    periods = [None for _ in range(len(tables))]
    starts = [None for _ in range(len(tables))]
    ends = [None for _ in range(len(tables))]
    durationtime = [None for _ in range(len(tables))]
    teams = [None for _ in range(len(tables))]
    i = 0
    while i < len(tables):
        # A convenient artefact of this package: search for [p, p, p, p, p, p, p, p]
        if len(tables[i]) == 8 and helpers.check_number_last_first_format(tables[i][0]):
            pname = helpers.remove_leading_number(tables[i][0])
            pname = helpers.flip_first_last(pname)
            pid = players.player_as_id(pname, teamid)
            i += 2  # skip the header row
            while re.match('\d{1,2}', tables[i][0]):  # First entry is shift number
                # print(tables[i])
                shiftnum, per, start, end, dur, ev = tables[i]
                # print(pname, pid, shiftnum, per, start, end)
                ids[i] = pid
                if per == 'OT':
                    per = 4
                periods[i] = int(per)
                starts[i] = start[:start.index('/')].strip()
                ends[i] = end[:end.index('/')].strip()
                durationtime[i] = helpers.mmss_to_secs(dur)
                teams[i] = teamid
                i += 1
            i += 1
        else:
            i += 1

    ids = [x for x in ids if x is not None]
    periods = [x for x in periods if x is not None]
    starts = [x for x in starts if x is not None]
    ends = [x for x in ends if x is not None]
    durationtime = [x for x in durationtime if x is not None]
    teams = [x for x in teams if x is not None]

    startmin = [x[:x.index(':')] for x in starts]
    startsec = [x[x.index(':') + 1:] for x in starts]
    starttimes = [1200 * (p - 1) + 60 * int(m) + int(s) + 1 for p, m, s in zip(periods, startmin, startsec)]
    # starttimes = [0 if x == 1 else x for x in starttimes]
    endmin = [x[:x.index(':')] for x in ends]
    endsec = [x[x.index(':') + 1:] for x in ends]
    # There is an extra -1 in endtimes to avoid overlapping start/end
    endtimes = [1200 * (p - 1) + 60 * int(m) + int(s) for p, m, s in zip(periods, endmin, endsec)]

    durationtime = [e - s for s, e in zip(starttimes, endtimes)]

    df = pd.DataFrame({'PlayerID': ids, 'Period': periods, 'Start': starttimes, 'End': endtimes,
                       'Team': teams, 'Duration': durationtime})
    dflst.append(df)
dflst = pd.concat(dflst)

In [8]:
%%timeit 
parse_toi._finish_toidf_manipulations(dflst, season, game)

1 loop, best of 3: 3.71 s per loop


In [9]:
%lprun -f parse_toi._finish_toidf_manipulations parse_toi._finish_toidf_manipulations(dflst, season, game)

![alt text](finish_toidf_manipulations1.png "Finish manipulations1")

![alt text](finish_toidf_manipulations2.png "Finish manipulations2")

![alt text](finish_toidf_manipulations3.png "Finish manipulations3")

![alt text](finish_toidf_manipulations4.png "Finish manipulations4")

![alt text](finish_toidf_manipulations5.png "Finish manipulations5")

It looks like the two lines that rank are the primary culprits here.

So is it the groupby, or the rank?

In [10]:
gameinfo = schedules.get_game_data_from_schedule(season, game)

# TODO don't read end times. Use duration, which has good coverage, to infer end. Then end + 1200 not needed below.
# Sometimes shifts have the same start and time.
# By the time we're here, they'll have start = end + 1
# So let's remove shifts with duration -1
df = df[df.Start != df.End + 1]

# Sometimes you see goalies with a shift starting in one period and ending in another
# This is to help in those cases.
if sum(df.End < df.Start) > 0:
    # ed.print_and_log('Have to adjust a shift time', 'warn')
    # TODO I think I'm making a mistake with overtime shifts--end at 3900!
    # TODO also, maybe only go to the end of the period, not to 1200
    # ed.print_and_log(df[df.End < df.Start])
    df.loc[df.End < df.Start, 'End'] = df.loc[df.End < df.Start, 'End'] + 1200
# One issue coming up is when the above line comes into play--missing times are filled in as 0:00
tempdf = df[['PlayerID', 'Start', 'End', 'Team', 'Duration']].query("Duration > 0")
tempdf = tempdf.assign(Time=tempdf.Start)
# print(tempdf.head(20))

# Let's filter out goalies for now. We can add them back in later.
# This will make it easier to get the strength later
pids = players.get_player_attrs('PlayerID', 'Pos')
tempdf = tempdf.merge(pids[['PlayerID', 'Pos']], how='left', on='PlayerID')

# toi = pd.DataFrame({'Time': [i for i in range(0, max(df.End) + 1)]})
toi = pd.DataFrame({'Time': [i for i in range(0, int(round(max(df.End))))]})

# Create one row per second
alltimes = toi.Time
newdf = pd.DataFrame(index=alltimes)

# Add rows and set times to True simultaneously
for i, (pid, start, end, team, duration, time, pos) in tempdf.iterrows():
    newdf.loc[start:end, pid] = True

# Fill NAs to False
for col in newdf:
    newdf.loc[:, col] = newdf[col].fillna(False)

# Go wide to long and then drop unneeded rows
newdf = helpers.melt_helper(newdf.reset_index(), id_vars='Time', # value_vars=newdf.columns,  # cols with num colnames causing errors
                            var_name='PlayerID', value_name='OnIce')
newdf = newdf[newdf.OnIce].drop('OnIce', axis=1)
newdf = newdf.merge(tempdf.drop('Time', axis=1), how='left', on='PlayerID') \
    .query("Time <= End & Time >= Start")

# In case there were rows that were all missing, join onto TOI
tempdf = toi.merge(newdf, how='left', on='Time')

goalies = tempdf[tempdf.Pos == 'G'].drop({'Pos'}, axis=1)
tempdf = tempdf[tempdf.Pos != 'G'].drop({'Pos'}, axis=1)

# Append team name to start of columns by team
home = str(gameinfo['Home'])
road = str(gameinfo['Road'])

# Goalies
# Let's assume we get only one goalie per second per team.
# TODO: flag if there are multiple listed and pick only one
goalies.loc[:, 'GTeam'] = goalies.Team.apply(lambda x: 'HG' if str(int(x)) == home else 'RG')
try:
    goalies2 = goalies[['Time', 'PlayerID', 'GTeam']] \
        .pivot(index='Time', columns='GTeam', values='PlayerID') \
        .reset_index()
except ValueError:
    # Duplicate entries in index error.
    # ed.print_and_log('Multiple goalies for a team in {0:d} {1:d}, picking one with the most TOI'.format(
    #    season, game), 'warn')

    # Find times with multiple goalies
    too_many_goalies_h = goalies[goalies.GTeam == 'HG'][['Time']] \
        .assign(GoalieCount=1) \
        .groupby('Time').count() \
        .reset_index() \
        .query('GoalieCount > 1')

    too_many_goalies_r = goalies[goalies.GTeam == 'RG'][['Time']] \
        .assign(GoalieCount=1) \
        .groupby('Time').count() \
        .reset_index() \
        .query('GoalieCount > 1')

    # Find most common goalie for each team
    if len(too_many_goalies_h) == 0:
        problem_times_revised_h = goalies
    else:  # i.e. if len(too_many_goalies_h) > 0:
        top_goalie_h = goalies[goalies.GTeam == 'HG'][['PlayerID']] \
            .assign(GoalieCount=1) \
            .groupby('PlayerID').count() \
            .reset_index() \
            .sort_values('GoalieCount', ascending=False) \
            .PlayerID.iloc[0]
        # and now finally drop problem times
        problem_times_revised_h = goalies \
            .merge(too_many_goalies_h[['Time']], how='outer', on='Time', indicator=True)
        problem_times_revised_h.loc[:, 'ToDrop'] = (problem_times_revised_h._merge == 'both') & \
                                                   (problem_times_revised_h.PlayerID != top_goalie_h)
        problem_times_revised_h = problem_times_revised_h[problem_times_revised_h.ToDrop == False] \
            .drop({'_merge', 'ToDrop'}, axis=1)

    if len(too_many_goalies_r) == 0:
        problem_times_revised_r = problem_times_revised_h
    else:  # i.e. if len(too_many_goalies_r) > 0:
        top_goalie_r = goalies[goalies.GTeam == 'RG'][['PlayerID']] \
            .assign(GoalieCount=1) \
            .groupby('PlayerID').count() \
            .reset_index() \
            .sort_values('GoalieCount', ascending=False) \
            .PlayerID.iloc[0]
        problem_times_revised_r = problem_times_revised_h \
            .merge(too_many_goalies_r[['Time']], how='outer', on='Time', indicator=True)
        problem_times_revised_r.loc[:, 'ToDrop'] = (problem_times_revised_r._merge == 'both') & \
                                                   (problem_times_revised_r.PlayerID != top_goalie_r)
        problem_times_revised_r = problem_times_revised_r[problem_times_revised_r.ToDrop == False] \
            .drop({'_merge', 'ToDrop'}, axis=1)

    # Pivot again
    goalies2 = problem_times_revised_r[['Time', 'PlayerID', 'GTeam']] \
        .pivot(index='Time', columns='GTeam', values='PlayerID') \
        .reset_index()

# Home
hdf = tempdf.query('Team == "' + home + '"').sort_values(['Time', 'Duration'], ascending=[True, False])
if len(hdf) == 0:
    # Earlier versions of pandas can have diff behavior
    hdf = tempdf.query('Team == ' + home).sort_values(['Time', 'Duration'], ascending=[True, False])

In [11]:
%timeit hdf2 = hdf[['Time', 'Duration']].groupby('Time')

1000 loops, best of 3: 588 µs per loop


In [12]:
%timeit hdf2 = hdf[['Time', 'Duration']].groupby('Time').rank(method='first', ascending=False)

100 loops, best of 3: 2.11 ms per loop


...These numbers don't really line up with the above...